In [7]:
import pandas as pd
# import ydata_profiling as pp
from pycaret.regression import *
pd.set_option('display.max_columns', None)

In [8]:
dataset = pd.read_csv("data_properti_filtered.csv")
dataset.head()

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,1300000000,70,102,2.0,1,0,Banten,Tangerang Selatan,BSD City,1
1,1500000000,60,64,2.0,2,0,Banten,Tangerang Selatan,Ciputat Timur,1
2,800000000,180,125,3.0,2,0,Banten,Tangerang,Karawaci,1
3,1500000000,130,90,3.0,2,0,Banten,Tangerang Selatan,Pondok Aren,1
4,2350000000,94,84,3.0,3,0,Banten,Tangerang Selatan,BSD City,1


In [9]:
prov = dataset[dataset['province'] == 'Banten']
prov['city'].value_counts()

city
Tangerang Selatan    21385
Tangerang            13441
Serang                 249
Lebak                  135
Cilegon                 56
Pandeglang              15
Name: count, dtype: int64

In [10]:
data_train = dataset.sample(frac=0.8, random_state=110)
data_unseen = dataset.drop(data_train.index)

data_train.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for train: ' + str(data_train.shape))
print('Data for test: ' +str(data_unseen.shape))

Data for train: (28225, 10)
Data for test: (7056, 10)


In [11]:
cat_features = ['province', 'city']
exp = setup(data_train, target = 'price', categorical_features=cat_features, 
            ignore_features = ['z', 'district', 'latitude', 'longitude'], 
            numeric_imputation = 'mean', categorical_imputation= 'mode', 
            remove_outliers= True)

,Description,Value
0,Session id,7322
1,Target,price
2,Target type,Regression
3,Original data shape,"(28225, 10)"
4,Transformed data shape,"(27237, 14)"
5,Transformed train set shape,"(18769, 14)"
6,Transformed test set shape,"(8468, 14)"
7,Ignore features,4
8,Numeric features,6
9,Categorical features,2


In [12]:
training_data = get_config(variable='X_train')
training_data

,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,cluster
27387,50,60,3.0,2,1,Banten,Tangerang Selatan,1
15130,145,216,4.0,4,0,Banten,Tangerang Selatan,1
20515,34,72,2.0,1,0,Banten,Tangerang,1
23292,200,147,5.0,2,0,Banten,Tangerang Selatan,1
9139,96,73,3.0,2,0,Banten,Tangerang Selatan,1
...,...,...,...,...,...,...,...,...
22912,52,121,2.0,2,0,Banten,Tangerang,1
10036,500,200,3.0,3,1,Banten,Tangerang,1
15440,36,72,2.0,1,0,Banten,Tangerang Selatan,1
6630,140,200,3.0,3,1,Banten,Tangerang Selatan,1


In [18]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,704153803.6452,2589792375589956608.0000,1594325115.0118,0.7847,0.4163,3.1389,0.4580
rf,Random Forest Regressor,624148266.0306,2731958488802779136.0000,1636616229.1520,0.7744,0.3849,2.0561,2.6170
et,Extra Trees Regressor,643266402.9070,3038485510781442560.0000,1730070751.3244,0.7460,0.3960,3.0867,2.1590
catboost,CatBoost Regressor,716060817.9238,3136553239224337920.0000,1750214011.3471,0.7384,0.4171,3.0306,3.4100
gbr,Gradient Boosting Regressor,792615636.4749,3187387523941516288.0000,1761585052.5863,0.7335,0.4397,6.5362,0.7210
xgboost,Extreme Gradient Boosting,724659150.2311,3548099000888645120.0000,1872202951.1079,0.7025,0.4093,3.6409,0.3750
knn,K Neighbors Regressor,707368267.4254,3672388339152433664.0000,1900534692.6093,0.6930,0.4191,4.9207,0.2980
dt,Decision Tree Regressor,769638581.8821,5396708611364780032.0000,2274914875.3589,0.5558,0.4558,1.7968,0.3000
ada,AdaBoost Regressor,1661596096.2191,5814910513585735680.0000,2378787741.9140,0.5142,0.9189,5.7589,0.6200
dummy,Dummy Regressor,1890705248.1840,12067330251994847232.0000,3460424540.2138,-0.0116,0.9781,3.3973,0.3180


In [14]:
#evaluate precition recall curve 
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [15]:
predict_model(best_model, data = data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,707993507.0785,2521541146523850240.0000,1587936127.9736,0.7858,0.4293,1.3250


,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster,price,prediction_label
0,130,90,3.0,2,0,Banten,Tangerang Selatan,Pondok Aren,1,1500000000,1.647767e+09
1,88,72,3.0,2,0,Banten,Tangerang Selatan,Serpong,1,1400000000,1.286277e+09
2,88,84,3.0,2,0,Banten,Tangerang Selatan,BSD City,1,2000000000,1.415485e+09
3,243,267,6.0,5,0,Banten,Tangerang Selatan,Ciputat,1,5250000000,5.400289e+09
4,250,233,5.0,3,0,Banten,Tangerang Selatan,Serpong,1,3100000000,3.679577e+09
...,...,...,...,...,...,...,...,...,...,...,...
7051,1000,534,5.0,6,6,Banten,Tangerang Selatan,BSD City,1,15500000000,1.662014e+10
7052,235,204,4.0,5,0,Banten,Tangerang Selatan,BSD City,1,9914780000,5.977444e+09
7053,380,280,5.0,3,0,Banten,Tangerang Selatan,BSD City,1,4600000000,4.481864e+09
7054,185,120,4.0,3,0,Banten,Tangerang Selatan,BSD City,1,2300000000,2.640802e+09


In [16]:
finalize_model(best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['buildingSize', 'landSize',
                                             'bedRooms', 'bathRooms', 'garages',
                                             'cluster'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['province', 'city'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['province', 'city'],
                                    transformer=OneHotEncoder(cols=['province',
                                                                    'city'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('remove_outliers',
                 TransformerWrapper(transformer=RemoveOutliers(random_state=7322))),
                ('actual_estimator',
                 LGBMRegressor(n_jobs=-1, random_state=7322))])

In [17]:
save_model(best_model, 'property_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['buildingSize', 'landSize',
                                              'bedRooms', 'bathRooms', 'garages',
                                              'cluster'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['province', 'city'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['province', 'city'],
                                     transformer=OneHotEncoder(cols=['province',
                                                                     'city'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True)))